In [134]:
import numpy as np
from itertools import combinations, product

Формирование V_i вектора

In [135]:
def i_vector(set, col):
    if len(set) == 0:
        return np.ones(2 ** col, int)
    else:
        vec = []
        for bin_v in list(product([0, 1], repeat = col)):
            f = np.prod([(bin_v[i] + 1) % 2 for i in set])
            vec.append(f)
        return vec

Создаёт матрицу кода Рида-Маллера порядка r

In [136]:
def create_RM_matrix(r, m):
    combs = [set for size in range(r + 1) for set in combinations(range(m), size)]
    return np.array([i_vector(set, m) for set in combs])

Список бинарных слов длины m

In [137]:
def Vec_h(I, m):
    return [word for word in product([0, 1], repeat=m) if np.prod([(word[idx] + 1) % 2 for idx in I])]

Вектор с учётом параметра сдвига t

In [138]:
def i_t_vector(I, m, t):
    return np.ones(2 ** m, int) if not I else [
        np.prod([(word[j] + t[j] + 1) % 2 for j in I]) for word in product([0, 1], repeat = m)
    ]

Упорядочивание комбинаций индексов в мажоритарном декодировании

In [139]:
def sorting(m, r):
    return sorted(combinations(range(m), r), key=len)

Мажоритарное декодирование для кода Рида-Маллера

In [140]:
def major_decode(w, r, m, size):
    w_r = w.copy()
    Mi = np.zeros(size, dtype=int)
    maximum_weight = 2 ** (m - r - 1) - 1
    idx = 0

    for i in range(r, -1, -1):
        for J in sorting(m, i):
            max_count = 2 ** (m - i - 1)
            zeros, ones = 0, 0
            dop = [x for x in range(m) if x not in J]

            for t in Vec_h(J, m):
                V = i_t_vector(dop, m, t)
                c = np.dot(w_r, V) % 2
                zeros += (c == 0)
                ones += (c == 1)

            if zeros > maximum_weight and ones > maximum_weight:
                return None
            if zeros > max_count:
                Mi[idx] = 0
            elif ones > max_count:
                Mi[idx] = 1
                w_r = (w_r + i_vector(J, m)) % 2
            idx += 1

        if len(w_r) < maximum_weight:
            Mi[idx:idx + len(sorting(m, i + 1))] = 0
            break

    return Mi

Генерация кодового слова с указанным числом ошибок для проверки алгоритма декодирования

In [141]:
def mistakes(G, error_count, u):
    u = u.dot(G) % 2
    mistake_position = np.random.choice(len(u), size = error_count, replace = False)
    u[mistake_position] = (u[mistake_position] + 1) % 2
    return u

Эксперименты

In [142]:
G_matr = create_RM_matrix(2, 4)
print("Порождающая матрица кода Рида-Маллера:\n", G_matr,"\n")
print("Однократная ошибка\n")
u = np.array([1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1])
print("Исходное сообщение:", u)
error = mistakes(G_matr, 1, u)
print("Слово с ошибкой:", error)

word_decoded = major_decode(error, 2, 4, len(G_matr))
if word_decoded is None:
    print("Ошибка")
else:
    V2 = word_decoded.dot(G_matr) % 2
    print("u * G:", V2)
    print("Исправленное слово:", word_decoded,"\n")

print("Двукратная ошибка\n")
u = np.array([1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1])
print("Исходное сообщение:", u)
error = mistakes(G_matr, 2, u)
print("Слово с ошибкой:", error)
word_decoded = major_decode(error, 2, 4, len(G_matr))

if word_decoded is None:
   print("\n Ошибка не исправлена")
else:
   V2 = word_decoded.dot(G_matr) % 2
   print("u * G:", V2)
   print("Исправленное слово:", word_decoded)

Порождающая матрица кода Рида-Маллера:
 [[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0]
 [1 1 1 1 0 0 0 0 1 1 1 1 0 0 0 0]
 [1 1 0 0 1 1 0 0 1 1 0 0 1 1 0 0]
 [1 0 1 0 1 0 1 0 1 0 1 0 1 0 1 0]
 [1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0]
 [1 1 0 0 1 1 0 0 0 0 0 0 0 0 0 0]
 [1 0 1 0 1 0 1 0 0 0 0 0 0 0 0 0]
 [1 1 0 0 0 0 0 0 1 1 0 0 0 0 0 0]
 [1 0 1 0 0 0 0 0 1 0 1 0 0 0 0 0]
 [1 0 0 0 1 0 0 0 1 0 0 0 1 0 0 0]] 

Однократная ошибка

Исходное сообщение: [1 0 0 0 1 1 0 0 0 1 1]
Слово с ошибкой: [1 0 1 0 1 1 0 1 0 1 1 1 1 1 0 1]
u * G: [1 0 0 0 1 1 0 1 0 1 1 1 1 1 0 1]
Исправленное слово: [1 0 0 0 1 1 0 0 0 1 1] 

Двукратная ошибка

Исходное сообщение: [1 0 0 0 1 1 0 0 0 1 1]
Слово с ошибкой: [1 1 0 0 1 1 0 1 0 1 1 1 1 0 0 1]

 Ошибка не исправлена
